<a href="https://colab.research.google.com/github/pansplawik/eegParkinsonCognitiveDisorders/blob/master/thirdVersion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mne

Importowanie pakietów

In [ ]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import csv
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from mne.decoding import PSDEstimator
from autoreject import AutoReject
from mne.decoding import Vectorizer

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, StratifiedKFold, cross_val_predict
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
# Models
from sklearn import svm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

Ściezki plików

In [ ]:
labels = pd.read_csv('/content/drive/MyDrive/dyplom/dataset.csv', sep=';')
labels['PD'] = labels['PD'].replace(2, 1)
for a in labels['ID']:
  print(a)
path='/content/drive/MyDrive/dyplom/EEG'

Preprocesing danych

In [ ]:
def load_epochs_from_dir(path, labels):
    epochs_array = []
    liczba=1
    filenames = sorted(os.listdir(path))
    for filename, label in zip(filenames, labels['PD']):
        print(filename)
        if filename.endswith('.edf'):
            filepath = os.path.join(path, filename)
            raw = mne.io.read_raw_edf(filepath, preload=True)
            raw.info.set_montage('standard_1020')
            raw.info['bads'].append('Oz')
            annotations = raw.annotations
            for i in range(len(annotations)):
              if annotations.description[i] == "break":
                annotations.description[i] = "BAD" 
              elif annotations.description[i] == 'Break end':
                annotations.description[i] = "BAD"
              elif annotations.description[i] == 'Miesnie':
                annotations.description[i] = "BAD"
              elif annotations.description[i] == 'Break start':
                annotations.description[i] = "BAD"
              elif annotations.description[i] == 'Ruch':
                annotations.description[i] = "BAD"
            raw.set_annotations(annotations)
            raw.filter(1,40, fir_design='firwin')
            events = mne.make_fixed_length_events(raw, duration=1.0)
            reject = dict(     
              eeg=82e-6,       
              )
            epochs = mne.Epochs(raw, events, tmin=0, tmax=2, baseline=(0,2), detrend=1,preload=True,reject_by_annotation='BAD',reject=reject)
            epochs_array.append([epochs,label])
            # try:
            #   epochs = ar.fit_transform(epochs)
            #   epochs_array.append([epochs,label])  
            # except Exception as e:
            #   print("Wystąpił wyjątek podczas przetwarzania danych:", e) 
            del epochs
            del raw
            del annotations
    return epochs_array


In [ ]:
def load_epochs_from_dir(path, labels):
    epochs_array = []
    liczba=1
    filenames = sorted(os.listdir(path))
    for filename, label in zip(filenames, labels['PD']):
      print(filename)
      if filename.endswith('.edf'):
        filepath = os.path.join(path, filename)
        raw = mne.io.read_raw_edf(filepath, preload=True)
        raw.info.set_montage('standard_1020')
        raw.info['bads'].append('Oz')
        annotations = raw.annotations
        for i in range(len(annotations)):
            if annotations.description[i] == "break":
                annotations.description[i] = "BAD" 
            elif annotations.description[i] == 'Break end':
                annotations.description[i] = "BAD"
            elif annotations.description[i] == 'Miesnie':
                annotations.description[i] = "BAD"
            elif annotations.description[i] == 'Break start':
                annotations.description[i] = "BAD"
            elif annotations.description[i] == 'Ruch':
                annotations.description[i] = "BAD"
        raw.set_annotations(annotations)
        raw.filter(1, 40, fir_design='firwin')
        events = mne.make_fixed_length_events(raw, duration=1.0)
        unique_events, unique_indices = np.unique(events[:, 0], return_index=True)
        events = events[unique_indices]
        tmin = 0
        tmax = 2
        baseline = (0, 2)
        reject = dict(     
              eeg=80e-6,       
              )
        epochs = mne.Epochs(raw, events, event_id=None, tmin=tmin, tmax=tmax, baseline=baseline, preload=True,reject_by_annotation='BAD',reject=reject)
        epochs_array.append([epochs, label])
        
        del raw
        del annotations
    return epochs_array


PSD

In [80]:
def psds(epochs_array, labels):
    data = []
    for epochs, label in zip(epochs_array, labels):
        if epochs is not None and len(epochs) > 0:
            epochs_data = epochs.get_data()
            epo_spectrum = mne.time_frequency.psd_array_multitaper(epochs_data, 100, fmin=0, fmax=40, bandwidth=4, adaptive=True, normalization='length', output='power', n_jobs=-1, max_iter=100, verbose=True)
            psds_epoch, freqs = epo_spectrum
            for p in psds_epoch:
                p = 10 * np.log10(p)
                psds_mean = p.mean(0)
                psds_std = p.std(0)
                data.append([p, label])
            del psds_epoch
            del freqs
            del epo_spectrum
    return data


In [ ]:
data = load_epochs_from_dir('/content/drive/MyDrive/dyplom/EEG',labels)
df = pd.DataFrame(data, columns=['value', 'label'])
df.to_csv('data.csv', index=False)

In [ ]:
psd = psds([epochs for epochs, label in data], [label for epochs, label in data])
df = pd.DataFrame(psd, columns=['psds', 'label'])
df.to_csv('psd.csv', index=False)

Tworzenie zestawów

In [85]:
X_train, X_test, y_train, y_test = train_test_split(df['psds'], df['label'], test_size=0.1, random_state=42)
X_train_2d = np.array([x[0] for x in X_train]).reshape(len(X_train), -1)
X_test_2d = np.array([x[0] for x in X_test]).reshape(len(X_test), -1)
y_train=np.array(y_train)
y_train=y_train.transpose()
y_test=np.array(y_test)
y_test=y_test.transpose()

In [88]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report
# Tworzenie i trenowanie modelu SVM
clf_svm_pip = make_pipeline(Vectorizer(), StandardScaler(), SVC())
clf_svm_pip.fit(X_train_2d, y_train)

# Przewidywanie etykiet dla danych testowych
predictions_svm = clf_svm_pip.predict(X_test_2d)

# Raport klasyfikacji
report_svm = classification_report(y_test, predictions_svm, target_names=['0', '1'])
print('SVM Classification Report:\n{}'.format(report_svm))

# Dokładność modelu SVM
acc_svm = accuracy_score(y_test, predictions_svm)
print("Accuracy of SVM model: {}".format(acc_svm))

# Obliczanie precyzji, czułości i miary F1
precision_svm, recall_svm, fscore_svm, support_svm = precision_recall_fscore_support(y_test, predictions_svm, average='macro')
print('Precision: {0}, Recall: {1}, F1-score: {2}'.format(precision_svm, recall_svm, fscore_svm))

SVM Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.87      0.85      3595
           1       0.86      0.82      0.84      3528

    accuracy                           0.84      7123
   macro avg       0.85      0.84      0.84      7123
weighted avg       0.85      0.84      0.84      7123

Accuracy of SVM model: 0.8448687350835322
Precision: 0.8456614793838524, Recall: 0.8446397427770367, F1-score: 0.844706821918905


In [87]:
from sklearn.ensemble import RandomForestClassifier

clf = make_pipeline(
    StandardScaler(),
    Vectorizer(),
    RandomForestClassifier()  # Używamy Random Forest z domyślnymi parametrami
)
clf.fit(X_train_2d, y_train)

# Przewidywanie etykiet dla danych testowych
y_pred = clf.predict(X_test_2d)
acc = accuracy_score(y_test, y_pred)

print("Accuracy score: {}".format(acc))

Accuracy score: 0.9785202863961814


In [86]:
from sklearn.linear_model import LogisticRegression

clf = make_pipeline(
StandardScaler(),
Vectorizer(),
LogisticRegression() # Używamy regresji logistycznej z domyślnymi parametrami
)
clf.fit(X_train_2d, y_train)

#Przewidywanie etykiet dla danych testowych
y_pred = clf.predict(X_test_2d)
acc = accuracy_score(y_test, y_pred)

print("Accuracy score: {}".format(acc))

Accuracy score: 0.6998455706865085


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [89]:
from joblib import dump, load
dump(clf, 'oficjalnyRandomForest.joblib') 

['oficjalnyRandomForest.joblib']